We create a supplementary table that is an excel file with the following sheets:
- target - annotated for updating to moalmanac
- moalmanac - aneuploidy
- moalmanac - copy_number
- moalmanac - germline variant
- moalmanac - knockdown
- moalmanac - microsatellite stability
- moalmanac - mutational burden
- moalmanac - mutational signature
- moalmanac - neoantigen burden
- moalmanac - rearrangement
- moalmanac - silencing
- moalmanac - somatic variant

In [1]:
import openpyxl
import pandas as pd
import glob 

options = {}
options['strings_to_formulas'] = False
options['strings_to_urls'] = False

column_order = ['disease', 
                'context', 
                'oncotree_term', 
                'oncotree_code', 
                'therapy_name', 
                'therapy_strategy', 
                'therapy_type', 
                'therapy_sensitivity',
                'therapy_resistance', 
                'favorable_prognosis', 
                'predictive_implication',
                'description', 
                'preferred_assertion', 
                'source_type', 
                'citation', 
                'url', 
                'doi',
                'pmid', 
                'nct', 
                'last_updated', 
                'adverse_event_risk', 
                'feature_type', 
                'gene', 
                'chromosome', 
                'start_position', 
                'end_position', 
                'reference_allele', 
                'alternate_allele', 
                'cdna_change', 
                'protein_change', 
                'variant_annotation', 
                'exon', 
                'rsid', 
                'pathogenic', 
                'direction', 
                'cytoband',
                'gene1', 
                'gene2', 
                'rearrangement_type', 
                'classification', 
                'minimum_mutations', 
                'mutations_per_mb',
                #'minimum_neoantigens',
                'cosmic_signature_number',
                'cosmic_signature_version',
                'event', 
                'locus', 
                'status', 
                'technique'
]

dtype_map = {
    'aneuploidy': 'Aneuploidy',
    'copy_number': 'Copy Number',
    'germline_variant': 'Germline variant',
    'knockdown': 'Knockdown',
    'microsatellite_stability': 'Microsatellite stability',
    'mutational_burden': 'Mutational burden',
    'mutational_signature': 'Mutational signature',
    'neoantigen_burden': 'Neoantigen burden',
    'rearrangement': 'Rearrangement',
    'silencing': 'Silencing',
    'somatic_variant': 'Somatic variant'
}

sheets = []
handles = glob.glob('../moalmanac-db/content/*')
for handle in sorted(handles):
    dtype = handle.split('/')[-1].split('.tsv')[0]
    feature_type = dtype_map[dtype]  
    tmp = pd.read_csv(handle, sep='\t')
    tmp['feature_type'] = feature_type
    sheets.append(tmp)


sort_columns = ['gene', 'gene1', 'gene2', 'therapy_sensitivity', 'therapy_resistance', 'favorable_prognosis', 'disease', 'therapy_name']
df = (pd
      .concat(sheets, ignore_index=True)
      .loc[:, column_order]
      .sort_values(sort_columns, ascending=True)
     )

In [2]:
filename = 'supplementary-table-2'
folder = f'../../../tables/supplementary-table-02'
!mkdir -p $folder
outname = f'{folder}/{filename}.xlsx'

writer = pd.ExcelWriter(outname)

target = pd.read_excel('../target/almanac-comparison.xlsx')
target.to_excel(writer, sheet_name='TableS2-TARGET', index=False)
df.to_excel(writer, sheet_name='TableS2-MOAlmanac', index=False)

writer.save()